### 1. What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with one million instances?

The depth of a well-balanced binary tree containing $n$ leaves is $log_2(n)$. So, if the Decision Tree is trained without restrictions, it will likely end up creating a nearly balanced binary tree, which will have a depth of $log_2(10^6) \approx 20$.

### 2. Is a node's Gini impurity generally lower or greater than its parent's? Is it generally lower/greater, or always lower/greater?

A node's Gini impurity is generally lower than its parent's. This is due to the CART training algorithm's cost function, which splits each node in a way that minimizes the weighted sum of its children's Gini impurities. However, it is possible for a node to have a higher Gini impurity than its parent, as long as this increase is more than compensated for by a decrease in the other

### 3. If a Decision Tree is overfitting the training set, is it a good idea to try decreasing max_depth?

If a Decision Tree is overfitting the training set, it may be a good idea to try decreasing max_depth, since this will constrain the model, regularizing it.

### 4. If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?

Decision Trees do not require feature scaling, so scaling the input features of a Decision Tree will not help with underfitting.

### 5. If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?

The computational complexity of training a Decision Tree is $O(n \times m \log(m))$. So, if we multiply the number of instances by 10, the training time will be multiplied by $K = \frac{10m \log(m)}{m \log(m)} = 10$. In other words, it will take roughly 10 hours.

### 6. If your training set contains 100,000 instances, will setting presort=True speed up training?

Setting presort=True will considerably slow down training, since presorting the training set requires $O(m \times n \log(n))$ operations. If the training set contains 100,000 instances, setting presort=True will result in $100,000 \times 100,000 \times \log(100,000) \approx 10^{10}$ operations, which will be much slower than training without presorting.

### 7. Train and fine-tune a Decision Tree for the moons dataset by following these steps:

a. Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.

In [2]:
from sklearn.datasets import make_moons
X,y = make_moons(n_samples=100, noise=0.15, random_state=42)

b. Use train_test_split() to split the dataset into a training set and a test set.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

c. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    'max_leaf_nodes': [10,20,50,100,200,500]
}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=3, verbose=2)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END ..................................max_leaf_nodes=10; total time=   0.0s
[CV] END ..................................max_leaf_nodes=10; total time=   0.0s
[CV] END ..................................max_leaf_nodes=10; total time=   0.0s
[CV] END ..................................max_leaf_nodes=20; total time=   0.0s
[CV] END ..................................max_leaf_nodes=20; total time=   0.0s
[CV] END ..................................max_leaf_nodes=20; total time=   0.0s
[CV] END ..................................max_leaf_nodes=50; total time=   0.0s
[CV] END ..................................max_leaf_nodes=50; total time=   0.0s
[CV] END ..................................max_leaf_nodes=50; total time=   0.0s
[CV] END .................................max_leaf_nodes=100; total time=   0.0s
[CV] END .................................max_leaf_nodes=100; total time=   0.0s
[CV] END .................................max_lea

In [7]:
best_params

{'max_leaf_nodes': 10}

d. Train it on the full training set using these hyperparameters, and measure your model's performance on the test set. You should get roughly 85% to 87% accuracy

In [8]:
best_tree = DecisionTreeClassifier(max_leaf_nodes=best_params['max_leaf_nodes'], random_state=42)
best_tree.fit(X_train, y_train)
accuracy = best_tree.score(X_test, y_test)
print("Test set accuracy: {:.2f}".format(accuracy))

Test set accuracy: 0.95


### 8. Grow a forest by following these steps:

a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn's ShuffleSplit class for this.

In [11]:
from sklearn.model_selection import ShuffleSplit

splitter = ShuffleSplit(n_splits=1000, test_size=0.2, random_state=42)
subsets = [split for split in splitter.split(X_train)]

b. Train one Decision Tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.

In [12]:
trees = []

for train_idx, _ in subsets:
    X_subset, y_subset = X_train[train_idx], y_train[train_idx]
    tree = DecisionTreeClassifier(max_leaf_nodes=best_params['max_leaf_nodes'], random_state=42)
    tree.fit(X_subset, y_subset)
    trees.append(tree)

c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy's mode() function for this). This gives you majority-vote predictions over the test set.

In [15]:
from scipy.stats import mode
import numpy as np

predictions = np.zeros((len(X_test), len(trees)))
for i, tree in enumerate(trees):
    predictions[:, i] = tree.predict(X_test)

majority_vote_predictions = mode(predictions, axis=1)[0].flatten()


d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!

In [16]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, majority_vote_predictions)
print(f"Random Forest accuracy: {accuracy * 100:.2f}%")

Random Forest accuracy: 95.00%
